## Querying local LLMs with Ollama

Ollama allows running large language models **locally** on your machine.
This removes any dependency on external APIs and keeps all data on-site.

In this notebook we use Ollama for:
- direct local queries
- streaming responses
- comparison with cloud-based models

We use `llama3` as the default model, but the same code works for
`mistral` or `deepseek-r1` if they are installed.

In [1]:
# ------------------------------------------------------------
# Select Ollama model
# ------------------------------------------------------------
OLLAMA_MODEL = "llama3"   # alternatives: "mistral", "deepseek-r1"

print("Using Ollama model:", OLLAMA_MODEL)

Using Ollama model: llama3


In [2]:
import ollama
from IPython.display import display, Markdown

response = ollama.chat(
    model=OLLAMA_MODEL,
    messages=[
        {"role": "system", "content": "You are a concise technical assistant."},
        {"role": "user", "content": "Explain self-attention mathematically."}
    ],
)

answer = response["message"]["content"]
display(Markdown(answer))

Self-attention is a key component of transformer models, and it's often misunderstood due to its complex mathematical formulation. Here's a step-by-step breakdown:

**Notations:**

* `Q`, `K`, and `V` are matrices representing queries, keys, and values, respectively.
* `n` is the sequence length (number of tokens in the input).
* `h` is the hidden size (dimensionality of the embeddings).

**Self-Attention Layer:**

1. **Query-Key Matrix Multiplication**: Compute the dot product between `Q` and `K` to obtain a matrix `A`:

$$A = \frac{Q K^T}{\sqrt{h}}$$

where `^T` denotes the transpose.

2. **Softmax Function**: Apply the softmax function to `A` element-wise, which normalizes the attention weights:

$$Attention(A) = softmax(A) = \frac{\exp(a_i)}{\sum_j \exp(a_j)}$$

where `a_i` is the `i`-th element of `A`.

3. **Weighted Sum**: Compute the weighted sum of the values matrix `V` using the attention weights:

$$Output = Concat(head_1, ..., head_k) W^O$$

where `head_i = Attention(A) V_i`, and `W^O` is a learnable weight matrix.

**Mathematical Representation:**

The self-attention layer can be represented mathematically as follows:

$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{h}}) \cdot V$$

This representation captures the attention mechanism's ability to weigh and combine the input values based on their similarity.

By breaking down this process into smaller components, you can better understand how self-attention works.

## Streaming responses with Ollama

Ollama also supports streaming output.
As with OpenAI, streaming delivers tokens incrementally and improves
interactivity for longer answers.

Again, streaming changes *how* results are delivered, not *what*
the model ultimately produces.

In [3]:
from IPython.display import display, Markdown

stream = ollama.chat(
    model=OLLAMA_MODEL,
    messages=[
        {"role": "system", "content": "You are a concise technical assistant."},
        {"role": "user", "content": "Explain self-attention mathematically."}
    ],
    stream=True,
)

accumulated = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    if "message" in chunk and "content" in chunk["message"]:
        accumulated += chunk["message"]["content"]
        display_handle.update(Markdown(accumulated))

Self-attention is a key component in Transformer models, which allows the model to weigh the importance of different input elements relative to each other.

Mathematically, self-attention can be represented as:

1. Query, Key, and Value matrices:
	* Q: Query matrix (shape: `(batch_size, sequence_length, embedding_dim)`), where `embedding_dim` is the dimensionality of the embeddings.
	* K: Key matrix (same shape as Q).
	* V: Value matrix (same shape as Q).

2. Dot Product Attention:
	* Compute attention scores for each input element using the dot product between query and key matrices:

`attention_scores = softmax(Q * K^T / sqrt(d))`

where `d` is the embedding dimension, and `softmax` is applied along the last axis (i.e., sequence length).

3. Weighted Sum:
	* Compute weighted sum of values based on attention scores:

`output = V * attention_scores`

The output is a matrix with shape `(batch_size, sequence_length, embedding_dim)`, where each row corresponds to an input element and its corresponding attention weights.

Note that the self-attention mechanism can be scaled by a learnable parameter `α` (alpha), which helps to regulate the overall magnitude of the attention weights. This can be incorporated into the computation as follows:

`output = V * alpha * attention_scores`

In practice, you would implement this using libraries like TensorFlow or PyTorch, where you can use built-in functions for matrix multiplications and softmax operations.